In [25]:
# Import necessary libraries
import ipywidgets as widgets
from IPython.display import display, HTML
# from LazyQML.lazyqml.lazyqml import QuantumClassifier
import pandas as pd
from sklearn.model_selection import train_test_split

# Add custom CSS for better aesthetics
display(HTML("""
<style>
    .widget-label {
        font-weight: bold;
        font-size: 14px;
    }
    .slider {
        width: 500px !important;
    }
    .widget-box {
        background-color: #f9f9f9;
        padding: 15px;
        border-radius: 10px;
        box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.1);
        margin-bottom: 15px;
    }
</style>
"""))

In [37]:

# Widgets for user input with enhanced styles and multi-selection
epochs_widget = widgets.IntSlider(
    min=1, max=1000, step=1, value=100, description="Epochs:",
    style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

layers_widget = widgets.IntSlider(
    min=1, max=100, step=1, value=5, description="Layers:",
    style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

nqubits_widget = widgets.IntSlider(
    min=1, max=20, step=1, value=8, description="Qubits:",
    style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

randomstate_widget = widgets.IntText(
    value=1234, description="Random State:",
    style={'description_width': '150px'}, layout=widgets.Layout(width='300px'))

# Classifier multi-selection using a Dropdown with multiple selection enabled
classifiers_options = ['qsvm', 'qnn', 'qnnbag']
classifiers_widget = widgets.SelectMultiple(
    options=classifiers_options, value=['qsvm'], description='Classifiers:',
    style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

# Ansatz multi-selection with Dropdown widget for intuitive multiple selection
ansatzs_options = ['HPzRx', 'tree_tensor', 'two_local', 'hardware_efficient']
ansatzs_widget = widgets.SelectMultiple(
    options=ansatzs_options, value=['HPzRx'], description='Ansatzs:',
    style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

# Embeddings multi-selection using Dropdown
embeddings_options = ['amplitude_embedding', 'ZZ_embedding', 'rx_embedding', 'rz_embedding', 'ry_embedding']
embeddings_widget = widgets.SelectMultiple(
    options=embeddings_options, value=['amplitude_embedding'], description='Embeddings:',
    style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

# Verbose switch
verbose_widget = widgets.ToggleButton(
    value=False, description="Verbose", icon='check',
    button_style='info', style={'description_width': '150px'}, layout=widgets.Layout(width='200px'))

# Number of samples as a percentage (from 0.0 to 1.0)
nsamples_widget = widgets.FloatSlider(
    value=1.0, min=0.0, max=1.0, step=0.01, description="Samples (0.0 - 1.0):",
    style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))


# Number of estimators (for QNN Bagging)
nestimators_widget = widgets.IntSlider(
    min=1, max=50, step=1, value=10, description="Estimators:",
    style={'description_width': '150px'}, layout=widgets.Layout(width='500px'))

# Function to display widgets in a nice box layout (stack widgets vertically)
def display_widgets():
    widget_box = widgets.VBox([
        widgets.HTML("<h3>Quantum Classifier Configuration</h3>"),
        epochs_widget,
        layers_widget,
        nqubits_widget,
        randomstate_widget,
        classifiers_widget,
        ansatzs_widget,
        embeddings_widget,
        widgets.HBox([verbose_widget, nsamples_widget]),
        nestimators_widget
    ], layout=widgets.Layout(border='solid 1px #cccccc', padding='20px', border_radius='10px'))
    
    display(widget_box)

display_widgets()


In [34]:
# Sample DataFrame input (can be customized by the user)
upload_button = widgets.FileUpload(accept=".csv", multiple=False)

def load_dataframe(change):
    if len(upload_button.value) > 0:
        # Read the uploaded CSV file content
        content = next(iter(upload_button.value.values()))['content']
        data = pd.read_csv(pd.compat.BytesIO(content))
        display(data.head())  # Show the first few rows of the dataframe
        return data
    else:
        return None

upload_button.observe(load_dataframe, names='value')
display(upload_button)

FileUpload(value=(), accept='.csv', description='Upload')

In [38]:
# Create an output widget to capture the print statements
output_display = widgets.Output()

# Function to train the model and display outputs
def train_model(data=None):
    try:
        print(f"Initializing QuantumClassifier with:\nVerbose: {verbose_widget.value}\nRandom State: {randomstate_widget.value}\nNumber of Qubits: {nqubits_widget.value}\nNumber of Layers: {layers_widget.value}\nClassifiers: {list(classifiers_widget.value)}\nEmbeddings: {list(embeddings_widget.value)}\nAnsatzs: {list(ansatzs_widget.value)}\nEpochs: {epochs_widget.value}\nSamples: {nsamples_widget.value}\nEstimators: {nestimators_widget.value}")
        # Split the dataset into train and test
        X = data.drop(columns=['target'])  # Assuming the target column is named 'target'
        y = data['target']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=randomstate_widget.value)

        

        # Initialize QuantumClassifier with selected parameters
        clf = QuantumClassifier(
            verbose=verbose_widget.value,
            randomSate=randomstate_widget.value,
            nqubits=nqubits_widget.value,
            numLayers=layers_widget.value,
            classifiers=list(classifiers_widget.value),
            ansatzs=list(ansatzs_widget.value),
            embeddings=list(embeddings_widget.value),
            epochs=epochs_widget.value,
            maxSamples=nsamples_widget.value,
            numPredictors=nestimators_widget.value
        )

        # Call the fit method and print the models and results
        models, predictions = clf.fit(X_train, X_test, y_train, y_test)
        display(models)  # Display the resulting models
    except:
        print("No data provided. Please upload a dataset.")



# Train button to trigger the training process
train_button = widgets.Button(description="Train Model", button_style='success')

def on_train_button_click(b):
    # Assuming data is loaded using the 'upload_button' widget
    data = load_dataframe(None)  # Call to load the dataframe
    train_model(data)

train_button.on_click(on_train_button_click)
display(train_button)


Button(button_style='success', description='Train Model', style=ButtonStyle())

Initializing QuantumClassifier with:
Verbose: True
Random State: 1234
Number of Qubits: 20
Number of Layers: 100
Classifiers: ['qsvm', 'qnn', 'qnnbag']
Embeddings: ['amplitude_embedding', 'ZZ_embedding', 'rx_embedding', 'rz_embedding', 'ry_embedding']
Ansatzs: ['HPzRx', 'tree_tensor', 'two_local', 'hardware_efficient']
Epochs: 1000
Samples: 0.8
Estimators: 10
No data provided. Please upload a dataset.
